In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
with open('amazon_in_urls.txt', 'r') as f:
    lines = [line.strip() for line in f]
    
reviewlist = []
domains = [".com", ".co.uk", ".ca", ".in", ".co.mx", ".com.au", ".co.jp", ".sg", ".nl", ".de", ".fr", ".it", ".es", ".com.br"]

In [ ]:
def get_soup(url):
    params = {'url': url, 'wait': 2}
    r = requests.get('http://localhost:8050/render.html', params=params)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [ ]:
def get_reviews(soup, product_name):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            product_title = product_name 
            review_date = item.find('span', {'data-hook': 'review-date'}).text.strip()
            review_date = ' '.join(review_date.split()[-3:]) #Reviewed in India on 15 July 2023

            rating_text = item.find('i', {'data-hook': 'review-star-rating'}).text.strip()
            rating_value = re.search(r'(\d)', rating_text) #4.0 out of 5 stars
            rating = int(rating_value)

            review = {
                'Product': product_title,
                'Rating': rating,
                'Date': review_date,
                'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
for i in range(0, len(lines), 2):
    base_url = lines[i]
    product_name = lines[i + 1]

    print(f"\nScraping reviews for: {product_name}")

    for y in domains:
        for x in range(1, 11):
            url = re.sub(r'www\.amazon\.com', f'www.amazon{y}', base_url)
            url = f'{url}&pageNumber={x}'
            soup = get_soup(url)
            print(f'Getting page: {x} from {y}')
            initial_length = len(reviewlist)
            get_reviews(soup, product_name)
            print(len(reviewlist))
            print(url)
            if len(reviewlist) == initial_length:
                break
            if not soup.find('li', {'class': 'a-disabled a-last'}):
                pass
            else:
                break

In [ ]:
output_file = '../data/amazon.csv'
df = pd.DataFrame(reviewlist)

if not df.empty:
    df.to_csv(output_file, index=False)

print('Fin.')

Fin.
